In [ ]:
# Setup session and imports
import streamlit as st
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Set context
session.sql("USE DATABASE CALL_CENTER_ANALYTICS").collect()
session.sql("USE SCHEMA AUDIO_PROCESSING").collect()
session.sql("USE WAREHOUSE AUDIO_CORTEX_WH").collect()

# Display dataset overview
email_stats = session.sql("""
    SELECT 
        COUNT(*) AS total_emails,
        COUNT(DISTINCT CUSTOMER_ID) AS unique_customers,
        MIN(DATE_RECEIVED) AS earliest_email,
        MAX(DATE_RECEIVED) AS latest_email
    FROM CUSTOMER_EMAILS
""").collect()[0]

col1, col2, col3, col4 = st.columns(4)
with col1:
    st.metric("📧 Total Emails", f"{email_stats['TOTAL_EMAILS']:,}")
with col2:
    st.metric("👥 Unique Customers", f"{email_stats['UNIQUE_CUSTOMERS']:,}")

In [ ]:
-- Generate executive summaries for customer emails (sample first to manage costs)
CREATE OR REPLACE TABLE EMAIL_SUMMARIES AS
SELECT 
    EMAIL_ID,
    CUSTOMER_ID,
    DATE_RECEIVED,
    EMAIL_CONTENTS,
    SNOWFLAKE.CORTEX.SUMMARIZE(EMAIL_CONTENTS) AS executive_summary,
    CURRENT_TIMESTAMP() AS processing_timestamp
FROM CUSTOMER_EMAILS
WHERE DATE_RECEIVED >= '2025-04-10'  -- Focus on recent emails for demo
LIMIT 50;  -- Sample for cost management during demo

-- Display the result
SELECT 
    EMAIL_ID,
    CUSTOMER_ID,
    DATE_RECEIVED,
    LEFT(EMAIL_CONTENTS, 150) || '...' AS email_preview,
    executive_summary
FROM EMAIL_SUMMARIES
ORDER BY DATE_RECEIVED DESC
LIMIT 10;


In [ ]:
-- Combined sentiment analysis and classification in one step
CREATE OR REPLACE TABLE EMAIL_SENTIMENT_CLASSIFICATION AS
SELECT 
    es.*,
    -- Sentiment Analysis
    SNOWFLAKE.CORTEX.SENTIMENT(EMAIL_CONTENTS) AS sentiment_score,
    CASE 
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(EMAIL_CONTENTS) >= 0.1 THEN '😊 Positive'
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(EMAIL_CONTENTS) <= -0.1 THEN '😞 Negative'
        ELSE '😐 Neutral'
    END AS sentiment_category,
    -- Classification
    AI_CLASSIFY(
        EMAIL_CONTENTS, 
        ['Complaint', 'Inquiry', 'Compliment', 'Technical Support', 'Billing Issue', 'General Information']
    ):labels[0]::STRING AS email_classification
FROM EMAIL_SUMMARIES es;

-- Display combined analysis results
SELECT 
    EMAIL_ID,
    CUSTOMER_ID,
    sentiment_score,
    sentiment_category,
    email_classification,
    LEFT(executive_summary, 100) || '...' AS summary_preview
FROM EMAIL_SENTIMENT_CLASSIFICATION
ORDER BY sentiment_score DESC
LIMIT 10;


In [ ]:
# Create comprehensive sentiment and classification visualization
st.markdown("### 📊 Customer Communication Analytics Dashboard")

# Get combined data
combined_df = session.sql("""
    SELECT 
        sentiment_category,
        email_classification,
        COUNT(*) as count,
        ROUND(AVG(sentiment_score), 3) as avg_score
    FROM EMAIL_SENTIMENT_CLASSIFICATION
    GROUP BY sentiment_category, email_classification
    ORDER BY count DESC
""").to_pandas()

# Sentiment distribution
sentiment_df = session.sql("""
    SELECT 
        sentiment_category,
        COUNT(*) as count,
        ROUND(AVG(sentiment_score), 3) as avg_score
    FROM EMAIL_SENTIMENT_CLASSIFICATION
    GROUP BY sentiment_category
    ORDER BY count DESC
""").to_pandas()

# Classification distribution
classification_df = session.sql("""
    SELECT 
        email_classification,
        COUNT(*) as count,
        ROUND(AVG(sentiment_score), 3) as avg_sentiment
    FROM EMAIL_SENTIMENT_CLASSIFICATION
    GROUP BY email_classification
    ORDER BY count DESC
""").to_pandas()

if not sentiment_df.empty and not classification_df.empty:
    # Create two-column layout for both visualizations
    col1, col2 = st.columns(2)
    
    with col1:
        st.markdown("#### 💭 Sentiment Distribution")
        
        # Snowflake brand colors for sentiment
        sentiment_colors = {
            '😊 Positive': '#10B981',    # Modern green
            '😐 Neutral': '#29B5E8',     # Snowflake blue
            '😞 Negative': '#EF4444'     # Modern red
        }
        
        colors = [sentiment_colors.get(cat, '#64748B') for cat in sentiment_df['SENTIMENT_CATEGORY']]
        
        # Create sentiment pie chart
        fig1, ax1 = plt.subplots(figsize=(8, 6))
        wedges, texts, autotexts = ax1.pie(
            sentiment_df['COUNT'], 
            labels=sentiment_df['SENTIMENT_CATEGORY'],
            autopct='%1.1f%%',
            colors=colors,
            startangle=90,
            textprops={'fontsize': 10, 'fontweight': 'bold'},
            wedgeprops={'edgecolor': 'white', 'linewidth': 2}
        )
        ax1.set_title('Customer Sentiment', fontsize=14, fontweight='bold', color='#1E293B', pad=15)
        ax1.axis('equal')
        st.pyplot(fig1, clear_figure=True)
        
        # Sentiment metrics
        for idx, row in sentiment_df.iterrows():
            st.metric(
                label=f"{row['SENTIMENT_CATEGORY']}",
                value=f"{row['COUNT']} emails",
                delta=f"Avg: {row['AVG_SCORE']}"
            )
    
    with col2:
        st.markdown("#### 🏷️ Communication Types")
        
        # Classification colors (using a professional palette)
        classification_colors = ['#29B5E8', '#10B981', '#EF4444', '#F59E0B', '#8B5CF6', '#06B6D4']
        
        # Create classification pie chart
        fig2, ax2 = plt.subplots(figsize=(8, 6))
        wedges, texts, autotexts = ax2.pie(
            classification_df['COUNT'], 
            labels=classification_df['EMAIL_CLASSIFICATION'],
            autopct='%1.1f%%',
            colors=classification_colors[:len(classification_df)],
            startangle=90,
            textprops={'fontsize': 10, 'fontweight': 'bold'},
            wedgeprops={'edgecolor': 'white', 'linewidth': 2}
        )
        ax2.set_title('Communication Categories', fontsize=14, fontweight='bold', color='#1E293B', pad=15)
        ax2.axis('equal')
        st.pyplot(fig2, clear_figure=True)
        
        # Classification metrics
        for idx, row in classification_df.iterrows():
            st.metric(
                label=f"{row['EMAIL_CLASSIFICATION']}",
                value=f"{row['COUNT']} emails",
                delta=f"Avg Sentiment: {row['AVG_SENTIMENT']}"
            )
    
    # Combined insights
    st.markdown("---")
    st.markdown("#### 🔍 Key Insights")
    
    col_insight1, col_insight2 = st.columns(2)
    
    with col_insight1:
        total_emails = sentiment_df['COUNT'].sum()
        positive_pct = (sentiment_df[sentiment_df['SENTIMENT_CATEGORY'] == '😊 Positive']['COUNT'].sum() / total_emails * 100) if total_emails > 0 else 0
        st.info(f"💡 **Sentiment**: {positive_pct:.1f}% of communications show positive sentiment")
    
    with col_insight2:
        top_category = classification_df.iloc[0]['EMAIL_CLASSIFICATION'] if len(classification_df) > 0 else "N/A"
        top_count = classification_df.iloc[0]['COUNT'] if len(classification_df) > 0 else 0
        st.info(f"📈 **Top Category**: {top_category} ({top_count} emails)")
    
    # Show combined sentiment-classification breakdown
    if not combined_df.empty:
        st.markdown("#### 📋 Detailed Breakdown: Sentiment by Category")
        
        # Create a pivot table for better display
        pivot_df = combined_df.pivot_table(
            index='EMAIL_CLASSIFICATION', 
            columns='SENTIMENT_CATEGORY', 
            values='COUNT', 
            fill_value=0
        )
        
        st.dataframe(pivot_df, use_container_width=True)

else:
    st.error("No data found. Please run the previous analysis steps.")


In [ ]:
-- Add intelligent filtering for escalation and priority identification
CREATE OR REPLACE TABLE EMAIL_PRIORITY_ANALYSIS AS
SELECT 
    esc.*,
    AI_FILTER(PROMPT('Does this customer email indicate the request needs escalating? {0}', EMAIL_CONTENTS)) AS needs_escalation,
    AI_FILTER(PROMPT('Is this customer expressing dissatisfaction or frustration? {0}', EMAIL_CONTENTS)) AS high_priority_email
FROM EMAIL_SENTIMENT_CLASSIFICATION esc;

-- Display priority analysis results
SELECT 
    EMAIL_ID,
    email_classification,
    sentiment_category,
    needs_escalation,
    high_priority_email,
    CASE 
        WHEN needs_escalation AND high_priority_email THEN '🔴 URGENT'
        WHEN needs_escalation OR high_priority_email THEN '🟡 PRIORITY'
        ELSE '🟢 STANDARD'
    END AS priority_level
FROM EMAIL_PRIORITY_ANALYSIS;


In [ ]:
-- Add intelligent filtering for escalation and priority identification
CREATE OR REPLACE TABLE EMAIL_PRIORITY_ANALYSIS AS
SELECT 
    esc.*,
    AI_FILTER(PROMPT('Does this customer email indicate the request needs escalating? {0}', EMAIL_CONTENTS)) AS needs_escalation,
    AI_FILTER(PROMPT('Is this customer expressing dissatisfaction or frustration? {0}', EMAIL_CONTENTS)) AS high_priority_email
FROM EMAIL_SENTIMENT_CLASSIFICATION esc;

-- Display priority analysis results
SELECT 
    EMAIL_ID,
    email_classification,
    sentiment_category,
    needs_escalation,
    high_priority_email,
    CASE 
        WHEN needs_escalation AND high_priority_email THEN '🔴 URGENT'
        WHEN needs_escalation OR high_priority_email THEN '🟡 PRIORITY'
        ELSE '🟢 STANDARD'
    END AS priority_level
FROM EMAIL_PRIORITY_ANALYSIS;


In [ ]:
-- Extract comprehensive customer intelligence using AI_COMPLETE with structured output
CREATE OR REPLACE TABLE COMPREHENSIVE_EMAIL_INTELLIGENCE AS
SELECT 
    epa.EMAIL_ID,
    epa.EMAIL_CONTENTS,
    epa.executive_summary,
    epa.sentiment_score,
    epa.email_classification,
    AI_COMPLETE(
        model => 'snowflake-arctic',
        prompt => 'Extract comprehensive customer service analytics from this customer email. Focus on actionable business insights that would compete with GONG platform capabilities: ' || epa.EMAIL_CONTENTS,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'communication_summary': {'type': 'string', 'description': 'Brief overview of the entire customer communication'},
                    'key_topics_discussed': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Main topics covered in the email'},
                    'customer_sentiment': {'type': 'string', 'enum': ['very_positive', 'positive', 'neutral', 'negative', 'very_negative']},
                    'response_urgency': {'type': 'string', 'enum': ['immediate', 'within_24h', 'within_week', 'standard']},
                    'issue_resolved': {'type': 'string', 'enum': ['fully_resolved', 'partially_resolved', 'unresolved', 'follow_up_scheduled']},
                    'next_steps': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Specific action items identified'},
                    'follow_up_required': {'type': 'boolean'},
                    'escalation_needed': {'type': 'boolean'},
                    'customer_satisfaction_indicators': {'type': 'array', 'items': {'type': 'string'}},
                    'communication_outcome': {'type': 'string', 'enum': ['successful', 'unsuccessful', 'pending_resolution']},
                    'action_items_for_agent': {'type': 'array', 'items': {'type': 'string'}},
                    'business_impact': {'type': 'string', 'enum': ['high', 'medium', 'low']},
                    'competitive_mentions': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Any competitor mentions'},
                    'upsell_opportunities': {'type': 'array', 'items': {'type': 'string'}}
                },
                'required': ['communication_summary', 'key_topics_discussed', 'customer_sentiment', 'response_urgency', 'issue_resolved', 'next_steps', 'follow_up_required', 'escalation_needed']
            }
        }
    ) AS comprehensive_intelligence
FROM EMAIL_PRIORITY_ANALYSIS epa;

-- Display the comprehensive intelligence results
SELECT 
    EMAIL_ID,
    comprehensive_intelligence
FROM COMPREHENSIVE_EMAIL_INTELLIGENCE;


In [ ]:
-- Combination 1: AI_FILTER + AI_AGG - Analyze patterns in escalation-required emails
SELECT 
    'Escalation Pattern Analysis' AS analysis_type,
    AI_AGG(
        CONCAT('Email ID: ', cei.EMAIL_ID, ', Customer ID: ', cei.CUSTOMER_ID, ', Date: ', cei.DATE_RECEIVED, '. Summary: ', cei.executive_summary),
        'Analyze these escalation-required customer emails and identify common patterns, triggers, and recommendations for preventing future escalations. Provide actionable insights for customer service management.'
    ) AS escalation_insights
FROM COMPREHENSIVE_EMAIL_INTELLIGENCE cei
WHERE cei.comprehensive_intelligence:escalation_needed = TRUE;

-- Combination 2: AI_CLASSIFY + AI_FILTER - Find complaint emails with negative sentiment
SELECT 
    'Complaint Analysis' AS analysis_type,
    email_classification,
    sentiment_category,
    COUNT(*) AS email_count,
    AI_AGG(
        CONCAT('Customer ID: ', CUSTOMER_ID, ', Date: ', DATE_RECEIVED, '. Summary: ', executive_summary),
        'Summarize the key issues in these negative complaint emails and provide recommendations for improving customer satisfaction and reducing complaint volume.'
    ) AS complaint_improvement_recommendations
FROM COMPREHENSIVE_EMAIL_INTELLIGENCE
WHERE email_classification = 'Complaint'
  AND sentiment_category = '😞 Negative'
GROUP BY email_classification, sentiment_category;


In [ ]:
# Create interactive dashboard displaying our AI-powered customer service analytics
st.markdown("### 🚀 GONG-Competitive Customer Service Intelligence Dashboard")

# Get comprehensive intelligence data
intelligence_df = session.table('COMPREHENSIVE_EMAIL_INTELLIGENCE').to_pandas()

if not intelligence_df.empty:
    # Summary metrics
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric("📧 Total Emails Analyzed", len(intelligence_df))
    
    with col2:
        avg_sentiment = intelligence_df['SENTIMENT_SCORE'].mean()
        st.metric("😊 Avg Sentiment Score", f"{avg_sentiment:.2f}")
    
    with col3:
        # Count emails by classification
        classification_counts = intelligence_df['EMAIL_CLASSIFICATION'].value_counts()
        top_category = classification_counts.index[0] if len(classification_counts) > 0 else "N/A"
        st.metric("🏷️ Top Email Type", top_category)
    
    with col4:
        # Parse JSON to count escalations needed
        escalation_count = 0
        for idx, row in intelligence_df.iterrows():
            try:
                intel_data = json.loads(row['COMPREHENSIVE_INTELLIGENCE'])
                if intel_data.get('escalation_needed', False):
                    escalation_count += 1
            except:
                pass
        st.metric("🚨 Escalations Needed", escalation_count)
    
    # Detailed email analysis
    st.markdown("### 📋 Detailed Customer Communication Intelligence")
    
    for idx, row in intelligence_df.iterrows():
        # Parse intelligence data to get customer info
        customer_name = "Unknown"
        customer_email = "Unknown"
        try:
            intel_data = json.loads(row['COMPREHENSIVE_INTELLIGENCE'])
            customer_name = intel_data.get('customer_name', 'Unknown')
            customer_email = intel_data.get('customer_email', 'Unknown')
        except:
            pass
        
        with st.expander(f"📧 {row['EMAIL_ID']} - {customer_name} ({row['EMAIL_CLASSIFICATION']})", expanded=False):
            # Customer Information Header
            st.markdown("**🔍 Customer Information**")
            customer_col1, customer_col2, customer_col3 = st.columns(3)
            with customer_col1:
                st.info(f"👤 **Customer:** {customer_name}")
            with customer_col2:
                st.info(f"📧 **Email:** {customer_email}")
            with customer_col3:
                st.info(f"📅 **Date:** {row['DATE_RECEIVED']}")
            
            st.markdown("---")  # Separator
            
            col_left, col_right = st.columns(2)
            
            with col_left:
                st.write("**📄 Executive Summary:**")
                st.write(row['EXECUTIVE_SUMMARY'])
                st.write(f"**😊 Sentiment:** {row['SENTIMENT_SCORE']:.2f}")
                st.write(f"**🏷️ Classification:** {row['EMAIL_CLASSIFICATION']}")
                st.write(f"**🆔 Customer ID:** {row['CUSTOMER_ID']}")
            
            with col_right:
                st.write("**🤖 AI Intelligence Analysis:**")
                try:
                    intel_data = json.loads(row['COMPREHENSIVE_INTELLIGENCE'])
                    
                    # Display key insights
                    if 'communication_summary' in intel_data:
                        st.write(f"**📋 Communication Summary:** {intel_data['communication_summary']}")
                    
                    if 'key_topics_discussed' in intel_data:
                        st.write(f"**🔑 Key Topics:** {', '.join(intel_data['key_topics_discussed'])}")
                    
                    if 'next_steps' in intel_data:
                        st.write(f"**➡️ Next Steps:** {', '.join(intel_data['next_steps'])}")
                    
                    if 'response_urgency' in intel_data:
                        urgency_emoji = {
                            'immediate': '🔴',
                            'within_24h': '🟡', 
                            'within_week': '🟠',
                            'standard': '🟢'
                        }
                        urgency = intel_data['response_urgency']
                        st.write(f"**⏰ Response Urgency:** {urgency_emoji.get(urgency, '⚪')} {urgency}")
                    
                    if 'business_impact' in intel_data:
                        impact_emoji = {'high': '🔴', 'medium': '🟡', 'low': '🟢'}
                        impact = intel_data['business_impact']
                        st.write(f"**💼 Business Impact:** {impact_emoji.get(impact, '⚪')} {impact}")
                    
                    if 'escalation_needed' in intel_data:
                        escalation = "Yes 🚨" if intel_data['escalation_needed'] else "No ✅"
                        st.write(f"**🚨 Escalation Needed:** {escalation}")
                    
                    if 'follow_up_required' in intel_data:
                        followup = "Yes 📞" if intel_data['follow_up_required'] else "No ✅"
                        st.write(f"**📞 Follow-up Required:** {followup}")
                        
                except json.JSONDecodeError:
                    st.write("Raw intelligence data:")
                    st.write(row['COMPREHENSIVE_INTELLIGENCE'])
    
    st.success("🎉 Advanced call center AI analytics completed! This demonstrates enterprise-grade conversation intelligence competing with GONG.")
    
else:
    st.error("No intelligence data found. Please run the previous analysis steps.")


In [ ]:
-- Create a flattened view of the comprehensive intelligence data
CREATE OR REPLACE VIEW FLATTENED_EMAIL_INTELLIGENCE AS
SELECT 
    -- Original columns
    EMAIL_ID,
    CUSTOMER_ID,
    DATE_RECEIVED,
    EMAIL_CONTENTS,
    executive_summary,
    sentiment_score,
    email_classification,
    
    -- Flattened JSON fields - Simple values
    comprehensive_intelligence:customer_name::STRING AS customer_name,
    comprehensive_intelligence:customer_email::STRING AS customer_email,
    comprehensive_intelligence:communication_summary::STRING AS communication_summary,
    comprehensive_intelligence:customer_sentiment::STRING AS ai_customer_sentiment,
    comprehensive_intelligence:response_urgency::STRING AS response_urgency,
    comprehensive_intelligence:issue_resolved::STRING AS issue_resolved,
    comprehensive_intelligence:business_impact::STRING AS business_impact,
    comprehensive_intelligence:communication_outcome::STRING AS communication_outcome,
    comprehensive_intelligence:follow_up_required::BOOLEAN AS follow_up_required,
    comprehensive_intelligence:escalation_needed::BOOLEAN AS escalation_needed,
    
    -- Array fields converted to comma-separated strings
    ARRAY_TO_STRING(
        PARSE_JSON(comprehensive_intelligence:key_topics_discussed), ', '
    ) AS key_topics_discussed,
    
    ARRAY_TO_STRING(
        PARSE_JSON(comprehensive_intelligence:next_steps), ', '
    ) AS next_steps,
    
    ARRAY_TO_STRING(
        PARSE_JSON(comprehensive_intelligence:action_items_for_agent), ', '
    ) AS action_items_for_agent,
    
    ARRAY_TO_STRING(
        PARSE_JSON(comprehensive_intelligence:customer_satisfaction_indicators), ', '
    ) AS customer_satisfaction_indicators,
    
    ARRAY_TO_STRING(
        PARSE_JSON(comprehensive_intelligence:competitive_mentions), ', '
    ) AS competitive_mentions,
    
    ARRAY_TO_STRING(
        PARSE_JSON(comprehensive_intelligence:upsell_opportunities), ', '
    ) AS upsell_opportunities,
    
    -- Array size counts for analytics
    ARRAY_SIZE(PARSE_JSON(comprehensive_intelligence:key_topics_discussed)) AS topic_count,
    ARRAY_SIZE(PARSE_JSON(comprehensive_intelligence:next_steps)) AS next_steps_count,
    ARRAY_SIZE(PARSE_JSON(comprehensive_intelligence:action_items_for_agent)) AS action_items_count,
    ARRAY_SIZE(PARSE_JSON(comprehensive_intelligence:upsell_opportunities)) AS upsell_opportunities_count
    
FROM COMPREHENSIVE_EMAIL_INTELLIGENCE;

-- Display the flattened data
SELECT * FROM FLATTENED_EMAIL_INTELLIGENCE LIMIT 5;


In [ ]:
-- Example 1: Explode key topics discussed into individual rows
CREATE OR REPLACE VIEW EXPLODED_KEY_TOPICS AS
SELECT 
    EMAIL_ID,
    CUSTOMER_ID,
    DATE_RECEIVED,
    comprehensive_intelligence:customer_name::STRING AS customer_name,
    comprehensive_intelligence:customer_email::STRING AS customer_email,
    comprehensive_intelligence:business_impact::STRING AS business_impact,
    f.value::STRING AS individual_topic
FROM COMPREHENSIVE_EMAIL_INTELLIGENCE,
LATERAL FLATTEN(input => PARSE_JSON(comprehensive_intelligence:key_topics_discussed)) f;

-- Example 2: Explode action items for agents into individual rows
CREATE OR REPLACE VIEW EXPLODED_ACTION_ITEMS AS
SELECT 
    EMAIL_ID,
    CUSTOMER_ID,
    DATE_RECEIVED,
    comprehensive_intelligence:customer_name::STRING AS customer_name,
    comprehensive_intelligence:response_urgency::STRING AS response_urgency,
    comprehensive_intelligence:escalation_needed::BOOLEAN AS escalation_needed,
    f.value::STRING AS individual_action_item,
    f.index AS action_item_sequence
FROM COMPREHENSIVE_EMAIL_INTELLIGENCE,
LATERAL FLATTEN(input => PARSE_JSON(comprehensive_intelligence:action_items_for_agent)) f;

-- Example 3: Explode upsell opportunities into individual rows
CREATE OR REPLACE VIEW EXPLODED_UPSELL_OPPORTUNITIES AS
SELECT 
    EMAIL_ID,
    CUSTOMER_ID,
    DATE_RECEIVED,
    comprehensive_intelligence:customer_name::STRING AS customer_name,
    comprehensive_intelligence:customer_email::STRING AS customer_email,
    comprehensive_intelligence:business_impact::STRING AS business_impact,
    f.value::STRING AS individual_upsell_opportunity,
    f.index AS opportunity_sequence
FROM COMPREHENSIVE_EMAIL_INTELLIGENCE,
LATERAL FLATTEN(input => PARSE_JSON(comprehensive_intelligence:upsell_opportunities)) f;

-- Display sample results
SELECT 'Key Topics Analysis' AS analysis_type, COUNT(*) AS total_topics 
FROM EXPLODED_KEY_TOPICS
UNION ALL
SELECT 'Action Items Analysis' AS analysis_type, COUNT(*) AS total_action_items 
FROM EXPLODED_ACTION_ITEMS
UNION ALL
SELECT 'Upsell Opportunities Analysis' AS analysis_type, COUNT(*) AS total_opportunities 
FROM EXPLODED_UPSELL_OPPORTUNITIES;


In [ ]:
-- Analytics Query 1: Customer Response Urgency Analysis
SELECT 
    response_urgency,
    COUNT(*) AS email_count,
    AVG(sentiment_score) AS avg_sentiment,
    SUM(CASE WHEN escalation_needed THEN 1 ELSE 0 END) AS escalation_count,
    SUM(CASE WHEN follow_up_required THEN 1 ELSE 0 END) AS followup_count,
    AVG(action_items_count) AS avg_action_items,
    AVG(upsell_opportunities_count) AS avg_upsell_opportunities
FROM FLATTENED_EMAIL_INTELLIGENCE
GROUP BY response_urgency
ORDER BY 
    CASE response_urgency 
        WHEN 'immediate' THEN 1 
        WHEN 'within_24h' THEN 2 
        WHEN 'within_week' THEN 3 
        WHEN 'standard' THEN 4 
        ELSE 5 
    END;

-- Analytics Query 2: Business Impact vs Customer Sentiment Analysis
SELECT 
    business_impact,
    ai_customer_sentiment,
    COUNT(*) AS email_count,
    AVG(sentiment_score) AS avg_snowflake_sentiment,
    AVG(topic_count) AS avg_topics_discussed,
    ROUND(AVG(upsell_opportunities_count), 2) AS avg_upsell_ops
FROM FLATTENED_EMAIL_INTELLIGENCE
WHERE business_impact IS NOT NULL AND ai_customer_sentiment IS NOT NULL
GROUP BY business_impact, ai_customer_sentiment
ORDER BY business_impact, ai_customer_sentiment;

-- Analytics Query 3: Top Discussion Topics Analysis
SELECT 
    individual_topic,
    COUNT(*) AS frequency,
    AVG(CASE WHEN business_impact = 'high' THEN 1 ELSE 0 END) AS high_impact_rate,
    COUNT(DISTINCT customer_name) AS unique_customers
FROM EXPLODED_KEY_TOPICS
WHERE individual_topic IS NOT NULL
GROUP BY individual_topic
HAVING COUNT(*) >= 2  -- Only topics mentioned multiple times
ORDER BY frequency DESC
LIMIT 10;

-- Analytics Query 4: Customer Escalation Patterns
SELECT 
    customer_name,
    customer_email,
    COUNT(*) AS total_emails,
    SUM(CASE WHEN escalation_needed THEN 1 ELSE 0 END) AS escalation_count,
    AVG(sentiment_score) AS avg_sentiment,
    MAX(DATE_RECEIVED) AS last_contact_date,
    SUM(action_items_count) AS total_action_items
FROM FLATTENED_EMAIL_INTELLIGENCE
WHERE customer_name IS NOT NULL
GROUP BY customer_name, customer_email
HAVING COUNT(*) > 1  -- Customers with multiple emails
ORDER BY escalation_count DESC, total_emails DESC;
